# 🥤 Saudi Arabian CSD Market Analysis

## Comprehensive Interactive Analysis Dashboard

**Dataset**: DUMMY DATA FOR PRECISION AREAS.xlsx  
**Period**: January - December 2024  
**Coverage**: 9 Regions, 13 Provinces, 90 Precision Areas

---

### 📋 Analysis Overview
This notebook provides comprehensive analysis of the Saudi Arabian Carbonated Soft Drink (CSD) market including:

1. **Data Loading & Preprocessing** - Data quality and preparation
2. **Exploratory Analysis** - Market overview and statistics  
3. **Geographic Intelligence** - Regional performance and opportunities
4. **Time Series Analysis** - Seasonal patterns and trends
5. **Strategic Insights** - Business opportunities and recommendations

---

## 1️⃣ Data Loading & Preprocessing

In [ ]:
# Import necessary libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import warnings
from datetime import datetime
import ipywidgets as widgets
from IPython.display import display, HTML

# Configure display settings
warnings.filterwarnings('ignore')
pd.set_option('display.max_columns', None)
pd.set_option('display.width', 1000)
plt.style.use('seaborn-v0_8')
sns.set_palette("husl")

print("📚 Libraries loaded successfully!")

In [ ]:
# Load the dataset
print("🔄 Loading dataset...")
file_path = "DUMMY DATA FOR PRECISION AREAS.xlsx"
df_original = pd.read_excel(file_path, sheet_name='Sheet6')

print(f"✅ Dataset loaded: {df_original.shape[0]:,} rows × {df_original.shape[1]} columns")
print(f"📅 Period: Jan 2024 - Dec 2024")
print(f"🌍 Coverage: {df_original['Region'].nunique()} regions, {df_original['Province'].nunique()} provinces")
print(f"🏭 Products: {df_original['BRAND'].nunique()} brands, {df_original['ITEM'].nunique()} SKUs")

In [ ]:
# Convert to long format for analysis
print("🔄 Converting to long format...")

# Identify monthly columns
monthly_cols = [col for col in df_original.columns if "'24" in col]

# Melt to long format
id_vars = ['Region', 'Province', 'Precision Area', 'MARKET', 'KEY MANU  & KINZA', 
          'BRAND', 'Brand-2', 'CSD & CSD +', 'CSD Flavor Segment', 'REG/DIET', 
          'KEY PACKS', 'SUB-BRAND', 'PACK TYPE', 'PACK SIZE', 'ITEM']

df_long = pd.melt(df_original, 
                  id_vars=id_vars,
                  value_vars=monthly_cols,
                  var_name='Month',
                  value_name='Sales')

# Clean and prepare time columns
df_long['Month'] = df_long['Month'].str.replace("'", "")
df_long['Date'] = pd.to_datetime(df_long['Month'], format='%b%y')
df_long = df_long.sort_values('Date')
df_long['Month_Name'] = df_long['Date'].dt.strftime('%B')
df_long['Month_Num'] = df_long['Date'].dt.month
df_long['Quarter'] = df_long['Date'].dt.quarter
df_long['Year'] = df_long['Date'].dt.year

# Add derived features
df_long['Sales_Millions'] = df_long['Sales'] / 1_000_000
df_long['Season'] = df_long['Month_Num'].map({
    12: 'Winter', 1: 'Winter', 2: 'Winter',
    3: 'Spring', 4: 'Spring', 5: 'Spring',
    6: 'Summer', 7: 'Summer', 8: 'Summer',
    9: 'Fall', 10: 'Fall', 11: 'Fall'
})
df_long['Ramadan_Period'] = df_long['Month_Num'].apply(lambda x: 1 if x == 10 else 0)

print(f"✅ Long format created: {df_long.shape[0]:,} records")
print(f"📊 Date range: {df_long['Date'].min().strftime('%Y-%m-%d')} to {df_long['Date'].max().strftime('%Y-%m-%d')}")

In [ ]:
# Data quality check
print("🔍 Data Quality Assessment:")

# Missing values
missing_values = df_long.isnull().sum()
if missing_values.sum() > 0:
    print(f"⚠️  Missing values found: {missing_values[missing_values > 0].to_dict()}")
else:
    print("✅ No missing values found")

# Duplicates
duplicates = df_long.duplicated().sum()
if duplicates > 0:
    print(f"⚠️  Found {duplicates:,} duplicate records")
    df_long = df_long.drop_duplicates()
    print(f"✅ Removed duplicates, now {df_long.shape[0]:,} records")
else:
    print("✅ No duplicate records found")

# Sales distribution
zero_sales = (df_long['Sales'] == 0).sum()
positive_sales = (df_long['Sales'] > 0).sum()
print(f"📈 Sales distribution: {positive_sales:,} positive ({positive_sales/len(df_long)*100:.1f}%), {zero_sales:,} zero ({zero_sales/len(df_long)*100:.1f}%)")

# Summary statistics
total_sales = df_long['Sales'].sum()
print(f"💰 Total market size: {total_sales/1_000_000:.1f}M SAR annually")

## 2️⃣ Market Overview & Key Metrics

In [ ]:
# Key market metrics dashboard
print("📊 KEY MARKET METRICS")
print("=" * 50)

# Overall market size
total_sales = df_long['Sales'].sum()
total_millions = total_sales / 1_000_000
avg_monthly_sales = total_sales / 12

print(f"💎 Total Annual Market: {total_millions:.2f}M SAR")
print(f"📅 Average Monthly: {avg_monthly_sales/1_000_000:.2f}M SAR")

# Geographic coverage
print(f"\n🌍 Geographic Coverage:")
print(f"   Regions: {df_long['Region'].nunique()}")
print(f"   Provinces: {df_long['Province'].nunique()}")
print(f"   Precision Areas: {df_long['Precision Area'].nunique()}")
print(f"   Markets: {df_long['MARKET'].nunique()}")

# Product portfolio
print(f"\n🥤 Product Portfolio:")
print(f"   Manufacturers: {df_long['KEY MANU  & KINZA'].nunique()}")
print(f"   Brands: {df_long['BRAND'].nunique()}")
print(f"   Flavors: {df_long['CSD Flavor Segment'].nunique()}")
print(f"   Pack Types: {df_long['PACK TYPE'].nunique()}")
print(f"   Pack Sizes: {df_long['PACK SIZE'].nunique()}")
print(f"   SKUs: {df_long['ITEM'].nunique()}")

In [ ]:
# Manufacturer market share
print("\n🏆 Manufacturer Market Share:")
manu_sales = df_long.groupby('KEY MANU  & KINZA')['Sales'].sum().sort_values(ascending=False)
total_sales = manu_sales.sum()

for manu, sales in manu_sales.items():
    share = (sales / total_sales) * 100
    status = "🥇" if share > 50 else "🥈" if share > 20 else "🥉" if share > 10 else "📊"
    print(f"   {status} {manu:15s}: {sales/1_000_000:6.2f}M ({share:5.1f}%)")

In [ ]:
# Top brands performance
print("\n🥤 Top 15 Brands by Sales:")
brand_sales = df_long.groupby('BRAND')['Sales'].sum().sort_values(ascending=False)

for i, (brand, sales) in enumerate(brand_sales.head(15).items(), 1):
    share = (sales / total_sales) * 100
    medal = "🥇" if i <= 3 else "🥈" if i <= 6 else "🥉" if i <= 10 else "📊"
    print(f"{medal} {i:2d}. {brand:20s}: {sales/1_000_000:6.2f}M ({share:4.1f}%)")

## 3️⃣ 📊 Interactive Visualizations

In [ ]:
# Monthly sales trend visualization
print("📈 Monthly Sales Trend Analysis")

monthly_total = df_long.groupby('Date')['Sales'].sum().reset_index()
monthly_total['Month_Name'] = monthly_total['Date'].dt.strftime('%B')
monthly_total['Sales_Millions'] = monthly_total['Sales'] / 1_000_000

# Create interactive plot
fig = px.line(monthly_total, 
              x='Month_Name', 
              y='Sales_Millions',
              title='📊 Saudi Arabian CSD Market - Monthly Sales Trend 2024',
              labels={'Month_Name': 'Month', 'Sales_Millions': 'Sales (Millions SAR)'},
              markers=True,
              line_shape='spline')

# Add annotations for peak and low months
peak_month = monthly_total.loc[monthly_total['Sales_Millions'].idxmax()]
low_month = monthly_total.loc[monthly_total['Sales_Millions'].idxmin()]

fig.add_annotation(x=peak_month['Month_Name'], y=peak_month['Sales_Millions'],
                   text=f"🔥 Peak: {peak_month['Sales_Millions']:.1f}M",
                   showarrow=True, arrowhead=2, arrowsize=1, arrowwidth=2,
                   arrowcolor="red", bgcolor="white", bordercolor="red")

fig.add_annotation(x=low_month['Month_Name'], y=low_month['Sales_Millions'],
                   text=f"❄️ Low: {low_month['Sales_Millions']:.1f}M",
                   showarrow=True, arrowhead=2, arrowsize=1, arrowwidth=2,
                   arrowcolor="blue", bgcolor="white", bordercolor="blue")

fig.update_layout(height=500, showlegend=False)
fig.show()

# Calculate seasonality metrics
seasonality_ratio = peak_month['Sales'] / low_month['Sales']
print(f"\n📊 Seasonality Analysis:")
print(f"   Peak Month: {peak_month['Month_Name']} ({peak_month['Sales_Millions']:.2f}M)")
print(f"   Lowest Month: {low_month['Month_Name']} ({low_month['Sales_Millions']:.2f}M)")
print(f"   Seasonality Ratio: {seasonality_ratio:.2f}x")

In [ ]:
# Regional performance visualization
print("🌍 Regional Performance Analysis")

regional_sales = df_long.groupby('Region')['Sales'].sum().sort_values(ascending=False)
regional_millions = regional_sales / 1_000_000

# Create interactive bar chart
fig = px.bar(x=regional_millions.index, 
              y=regional_millions.values,
              title='🌍 Saudi Arabia - Regional Sales Performance',
              labels={'x': 'Region', 'y': 'Sales (Millions SAR)'},
              text=regional_millions.values.round(1),
              color=regional_millions.values,
              color_continuous_scale='viridis')

fig.update_traces(texttemplate='%{text:.1f}M', textposition='outside')
fig.update_layout(height=500, showlegend=False)
fig.show()

# Market concentration analysis
regional_shares = regional_sales / regional_sales.sum()
hhi = (regional_shares ** 2).sum()
concentration = "HIGH" if hhi > 0.25 else "MODERATE" if hhi > 0.15 else "LOW"

print(f"\n📊 Geographic Concentration: {concentration} (HHI: {hhi:.3f})")

In [ ]:
# Flavor segment analysis
print("🍋 Flavor Segment Preferences")

flavor_sales = df_long.groupby('CSD Flavor Segment')['Sales'].sum().sort_values(ascending=False)
flavor_millions = flavor_sales / 1_000_000

# Create pie chart
fig = px.pie(values=flavor_millions.values,
              names=flavor_millions.index,
              title='🍋 Saudi Arabia - Flavor Segment Market Share',
              labels={'value': 'Sales (Millions SAR)', 'names': 'Flavor Segment'})

fig.update_traces(textposition='inside', textinfo='percent+label')
fig.update_layout(height=500)
fig.show()

print("\n📊 Flavor Performance:")
for flavor, sales in flavor_sales.items():
    share = (sales / flavor_sales.sum()) * 100
    print(f"   {flavor:15s}: {sales/1_000_000:6.2f}M ({share:5.1f}%)")

## 4️⃣ 🎯 Strategic Analysis & Insights

In [ ]:
# Distribution gaps analysis
print("🚨 Distribution Gap Analysis")
print("=" * 40)

zero_analysis = df_long[df_long['Sales'] == 0].copy()
total_records = len(df_long)
zero_records = len(zero_analysis)
zero_percentage = (zero_records / total_records) * 100

print(f"📊 Overall Distribution Gaps:")
print(f"   Zero Sales Records: {zero_records:,} ({zero_percentage:.1f}%)")
print(f"   Positive Sales Records: {total_records-zero_records:,} ({100-zero_percentage:.1f}%)")

# Geographic distribution gaps
print(f"\n🌍 Geographic Distribution Gaps:")
zero_by_region = zero_analysis.groupby('Region').size()
total_by_region = df_long.groupby('Region').size()
zero_pct_by_region = (zero_by_region / total_by_region * 100).sort_values(ascending=False)

print("   Top 5 Regions with Highest Zero Sales %:")
for region, pct in zero_pct_by_region.head(5).items():
    urgency = "🔴 CRITICAL" if pct > 60 else "🟡 HIGH" if pct > 50 else "🟠 MODERATE"
    print(f"   {urgency}: {region:20s} {pct:.1f}%")

# Pack type inefficiencies
print(f"\n📦 Pack Type Distribution Inefficiencies:")
zero_by_pack = zero_analysis.groupby('PACK TYPE').size()
total_by_pack = df_long.groupby('PACK TYPE').size()
zero_pct_by_pack = (zero_by_pack / total_by_pack * 100).sort_values(ascending=False)

for pack_type, pct in zero_pct_by_pack.items():
    status = "🔴 CRITICAL" if pct > 60 else "🟡 HIGH" if pct > 40 else "🟢 GOOD"
    print(f"   {status}: {pack_type:15s} {pct:.1f}% zero sales")

In [ ]:
# Diet segment opportunity analysis
print("🥤 Diet Segment Opportunity Analysis")
print("=" * 40)

diet_by_region = df_long[df_long['REG/DIET'] == 'DIET'].groupby('Region')['Sales'].sum()
total_by_region = df_long.groupby('Region')['Sales'].sum()
diet_penetration = (diet_by_region / total_by_region * 100).fillna(0).sort_values()

print("📊 Diet Segment Penetration by Region (Opportunity Areas):")
for region, penetration in diet_penetration.head(5).items():
    opportunity = "🔥 HIGH" if penetration < 5 else "🟡 MEDIUM" if penetration < 10 else "🟢 LOW"
    print(f"   {opportunity}: {region:20s} {penetration:.1f}% diet penetration")

# Overall diet vs regular
reg_diet_sales = df_long.groupby('REG/DIET')['Sales'].sum()
total_sales = reg_diet_sales.sum()
diet_share = (reg_diet_sales.get('DIET', 0) / total_sales) * 100

print(f"\n📊 Overall Diet/Regular Split:")
print(f"   Regular: {reg_diet_sales.get('REG', 0)/1_000_000:.2f}M ({(reg_diet_sales.get('REG', 0)/total_sales)*100:.1f}%)")
print(f"   Diet: {reg_diet_sales.get('DIET', 0)/1_000_000:.2f}M ({diet_share:.1f}%)")
print(f"\n🎯 Diet Market Opportunity: {'EXTREME' if diet_share < 10 else 'HIGH' if diet_share < 15 else 'MODERATE'}")

In [ ]:
# Ramadan impact analysis
print("🕌 Ramadan Impact Analysis")
print("=" * 30)

ramadan_data = df_long.groupby(['Ramadan_Period', 'Month_Num'])['Sales'].sum().reset_index()
ramadan_month = ramadan_data[ramadan_data['Ramadan_Period'] == 1]
non_ramadan_data = ramadan_data[ramadan_data['Ramadan_Period'] == 0]

if not ramadan_month.empty:
    ramadan_sales = ramadan_month['Sales'].iloc[0]
    avg_non_ramadan = non_ramadan_data['Sales'].mean()
    ramadan_impact = ((ramadan_sales - avg_non_ramadan) / avg_non_ramadan) * 100 if avg_non_ramadan > 0 else 0
    
    print(f"📊 Overall Ramadan Impact:")
    print(f"   Ramadan Sales: {ramadan_sales/1_000_000:.2f}M")
    print(f"   Average Non-Ramadan Month: {avg_non_ramadan/1_000_000:.2f}M")
    print(f"   Ramadan Impact: {ramadan_impact:+.1f}% vs average")
    
    # Regional Ramadan impact
    print(f"\n🌍 Regional Ramadan Impact:")
    regional_ramadan = df_long.groupby(['Region', 'Ramadan_Period'])['Sales'].sum().reset_index()
    regional_ramadan_pivot = regional_ramadan.pivot(index='Region', columns='Ramadan_Period', values='Sales').fillna(0)
    
    ramadan_impacts = []
    for region in regional_ramadan_pivot.index:
        ramadan_sales_region = regional_ramadan_pivot.loc[region, 1] if 1 in regional_ramadan_pivot.columns else 0
        non_ramadan_sales_region = regional_ramadan_pivot.loc[region, 0] if 0 in regional_ramadan_pivot.columns else 0
        
        if non_ramadan_sales_region > 0:
            ramadan_avg = ramadan_sales_region / 1
            non_ramadan_avg = non_ramadan_sales_region / 11
            impact = ((ramadan_avg - non_ramadan_avg) / non_ramadan_avg) * 100
            ramadan_impacts.append((region, impact))
    
    ramadan_impacts.sort(key=lambda x: x[1], reverse=True)
    print("   Regional Ramadan Impact (% vs average month):")
    for region, impact in ramadan_impacts:
        sensitivity = "🔥 HIGHLY SENSITIVE" if impact > 50 else "🟡 SENSITIVE" if impact > 25 else "🟢 MODERATE"
        print(f"   {sensitivity}: {region:20s} {impact:+6.1f}%")
else:
    print("No Ramadan period identified in the data")

## 5️⃣ 🚀 Strategic Recommendations Summary

In [ ]:
# Generate strategic recommendations
print("🎯 STRATEGIC RECOMMENDATIONS")
print("=" * 50)

# Calculate key metrics for recommendations
zero_pct = zero_percentage
diet_penetration_lowest = diet_penetration.min()
hhi_value = hhi
ramadan_impact_val = ramadan_impact if 'ramadan_impact' in locals() else 0

print("\n🚀 IMMEDIATE ACTIONS (0-3 months):")

if zero_pct > 40:
    print(f"   🔥 CRITICAL: Fix {zero_pct:.1f}% distribution gaps immediately")
    print(f"      Estimated Revenue Uplift: {zero_pct * 0.15:.1f}%")

if diet_penetration_lowest < 10:
    print(f"   🥤 LAUNCH: Diet segment expansion in low-penetration regions")
    print(f"      Target: {diet_penetration.idxmax()} ({diet_penetration_lowest:.1f}% penetration)")

print("\n📈 MEDIUM-TERM STRATEGIES (3-12 months):")

if hhi_value < 0.2:
    print(f"   🎯 ATTACK: Low market concentration ({hhi_value:.3f}) creates expansion opportunities")

if ramadan_impact_val > 25:
    print(f"   🕌 OPTIMIZE: High Ramadan sensitivity ({ramadan_impact_val:+.1f}%) requires special programming")

print("\n🎖️ LONG-TERM INITIATIVES (12+ months):")
print("   🔬 PREDICT: Advanced analytics for demand forecasting")
print("   🛡️ DEFEND: Market intelligence system for competitive monitoring")
print("   📊 OPTIMIZE: Data-driven portfolio management")

print("\n💰 ESTIMATED IMPACT:")
immediate_roi = (zero_pct * 0.15 + (10 - diet_penetration_lowest) * 0.2 + ramadan_impact_val * 0.05)
print(f"   Immediate Revenue Uplift: {immediate_roi:.1f}%")
print(f"   12-Month Strategic ROI: {immediate_roi * 2:.1f}%")
print(f"   Strategic Priority: {'HIGH' if zero_pct > 45 else 'MEDIUM'}")

---

## 📋 Analysis Complete!

### 🎯 Key Takeaways:

1. **Market Size**: The Saudi Arabian CSD market is worth **{total_millions:.1f}M SAR** annually
2. **Market Concentration**: **{concentration}** with HHI of **{hhi:.3f}**
3. **Distribution Gaps**: **{zero_percentage:.1f}%** of potential sales not being realized
4. **Diet Opportunity**: Diet segment severely underpenetrated at **{diet_share:.1f}%**
5. **Seasonality**: Strong seasonal patterns with **{seasonality_ratio:.1f}x** variation

### 🚀 Next Steps:

1. **Review the detailed analysis scripts** in the `/scripts/` folder
2. **Run the comprehensive analysis** using the modular Python modules
3. **Implement the strategic recommendations** based on your business priorities
4. **Set up monitoring** to track progress against identified KPIs

---

*This interactive notebook provides a comprehensive foundation for data-driven decision making in the Saudi Arabian CSD market.*